In [1]:
cd ..

/Users/supawat/Code/radio-gyms


In [8]:
!pip install tensorflow

     |████████████████████████████████| 217.4 MB 1.1 MB/s eta 0:00:01    |████▊                           | 32.2 MB 1.6 MB/s eta 0:01:54     |████████████████████████████▊   | 194.9 MB 1.4 MB/s eta 0:00:16
     |████████████████████████████████| 57 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 13.0 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 962 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 8

## Import Gym

In [7]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
from radio_gyms.gyms import LocationGym

In [4]:
MAP_SCENE = "./assets/models/poznan.obj"

## Agent

In [ ]:
class Critic(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.l1 = tf.keras.layers.Dense(state_N,activation='relu')
        self.l2 = tf.keras.layers.Dense(state_N/2,activation='relu')
        self.v = tf.keras.layers.Dense(1, activation = None)
        
    def call(self, input_):
        output_ = self.l1(input_)
        output_ = self.l2(output_)
        output_ = self.v(output_)
        return output_
    
class Actor(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.l1 = tf.keras.layers.Dense(state_N,activation='relu')
        self.l2 = tf.keras.layers.Dense(state_N/2,activation='relu')
        self.v = tf.keras.layers.Dense(action_N, activation = 'softmax')
        
    def call(self, input_):
        output_ = self.l1(input_)
        output_ = self.l2(output_)
        output_ = self.v(output_)
        return output_
    
class Agent():
    def __init__(self, gamma=0.99):
        self.gamma = gamma
        self.a_opt = tf.keras.optimizers.Adam(learning_rate=5e-6)
        self.c_opt = tf.keras.optimizers.Adam(learning_rate=5e-6)
        try:
            self.actor = tf.keras.models.load_model(actor_path, compile=False)
            self.critic = tf.keras.models.load_model(critic_path, compile=False)
            print("Loaded the saved model")
        except:
            print("Created the new model")
            self.actor = Actor()
            self.critic = Critic()
        
    def act(self, state):
        prob = self.actor(np.array([state]))
        print(prob)
        prob = prob.numpy()
        dist = tfp.distributions.Categorical(probs=prob, dtype=tf.float32)
        action = dist.sample()
        return int(action.numpy()[0])
    
    def actor_loss(self, prob, action, td):
        dist = tfp.distributions.Categorical(probs=prob, dtype=tf.float32)
        log_prob = dist.log_prob(action)
        loss = -log_prob*td
        return loss
    
    def learn(self, state, action, reward, next_state):
        state = np.array([state])
        next_state = np.array([next_state])
        with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            p = self.actor(state, training=True)
            v =  self.critic(state,training=True)
            vn = self.critic(next_state, training=True)
            td = reward + self.gamma*vn*(1-int(done)) - v
            a_loss = self.actor_loss(p, action, td)
            c_loss = td**2
        grads1 = tape1.gradient(a_loss, self.actor.trainable_variables)
        grads2 = tape2.gradient(c_loss, self.critic.trainable_variables)
        self.a_opt.apply_gradients(zip(grads1, self.actor.trainable_variables))
        self.c_opt.apply_gradients(zip(grads2, self.critic.trainable_variables))
        return a_loss, c_loss

## Train agent

In [ ]:
agent = Agent()

In [6]:
gym = LocationGym(MAP_SCENE)

In [ ]:
episode_n = 10
for ep in range(episode_n):
    done = False
    state = gym.get_states()
    total_reward = 0
    rewards = []
    while not done:
        actions = agent.act(state)
        gym.action(actions)
        gym.step()
        reward = gym.get_reward()
        next_states = gym.get_states()
        rewards.append(rewards)
        a_loss, c_loss = agent.learn(states, actions, reward, next_states)
        states = next_states    